In [1]:
!pip install snscrape
!pip install googletrans == 3.1.0a0
!pip install NRCLex
!pip install tweepy == 4.6.0
!pip install text2emotion
!pip install vaderSentiment
!pip install twittersentiment
!pip install flair


zsh:1: = not found
     |████████████████████████████████| 396 kB 2.1 MB/s eta 0:00:01
  Created wheel for NRCLex: filename=NRCLex-3.0.0-py3-none-any.whl size=43310 sha256=9d4978f615c46e7e01faa1448f6e7b9b55b208fa12ac0a01321dfd3c0a9c4be1
  Stored in directory: /home/blindedsoul/.cache/pip/wheels/83/95/c0/42b43fb15eb48e4f5a67cba8915540cb2783591c59c037a9e5
Successfully built NRCLex
zsh:1: = not found
  Using cached text2emotion-0.0.5-py3-none-any.whl (57 kB)
  Using cached emoji-1.7.0.tar.gz (175 kB)
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171029 sha256=3e39e02ba0e05ff7e1ff22882fef21c6d3b5f1fc692b97dca00343ea0846d555
  Stored in directory: /home/blindedsoul/.cache/pip/wheels/5e/8c/80/c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji
     |████████████████████████████████| 125 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 3.7 MB/s eta 0:00:01     |██████████████▋                 | 1.6 MB 3.7 MB/s eta 0:00:01
 

In [3]:
import snscrape.modules.twitter as sntwitter
import snscrape.modules.twitter as sninstagram
import os
import re
import pandas as pd
import json
import time
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from nrclex import NRCLex
from flair.models import TextClassifier
from flair.data import Sentence
import text2emotion as te
from googletrans import Translator
import matplotlib.pyplot as plt
from twittersentiment import TwitterSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')


[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/blindedsoul/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/blindedsoul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
pd.read_json("text-query-tweets.json", lines=True)


In [11]:
import os

# Using OS library to call CLI commands in Python
os.system("snscrape --jsonl --max-results 500 --since 2021-01-21 twitter-search 'paro contra rappi until:2022-03-28' > text-query-tweets.json")


0

# TEXTBLOB PROCESS

In [43]:
class TwitterScrapeSentimentAnalysis():
    
    def __init__(self):
        try:
            self.sinceDate = "2022-01-01"
            self.untilDate = "2022-03-13"
            self.hashtag = ""
        except:
            print("Error: scraper Failed")

    def read_file(self,hashtag, since, until):
        print("Reading file...")
        os.system("snscrape --jsonl --max-results 500 --since {since} twitter-hashtag '{hashtag} until:{until}' > text-query-tweets.json".format(hashtag=hashtag , since = since , until = until))
        print("Reading file...")
        tweets = pd.read_json("text-query-tweets.json", lines=True)
        return tweets

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())

    def get_sentiment_tweet(self, tweet):
        translator = Translator()
        tweet = self.clean_tweet(tweet)
        tweet_en = translator.translate(tweet,dest="en")
        blob = TextBlob(tweet_en.text,analyzer=NaiveBayesAnalyzer())
        return blob.sentiment

    def get_sentiment_tweet_pol_sub(self, tweet):
        translator = Translator()
        tweet = self.clean_tweet(tweet)
        tweet_en = translator.translate(tweet,dest="en")
        blob = TextBlob(tweet_en.text)
        return blob.sentiment

    def get_tweets(self):
        tweets = []
        print("Fetching tweets...")
        tweets_data = self.read_file(self.hashtag, self.sinceDate, self.untilDate)
        
        for i in range(len(tweets_data)):
            parsed_tweet = {}
            try :
                parsed_tweet['date'] = tweets_data.iloc[i]['date']
                parsed_tweet['conversationId'] = tweets_data.iloc[i]['conversationId']
                parsed_tweet['tweet'] = tweets_data.iloc[i]['content']
                aux_class_tweet = self.get_sentiment_tweet(
                    str(tweets_data.iloc[i]['content']))
                aux_class_tweet_pol_sub = self.get_sentiment_tweet_pol_sub(
                    str(tweets_data.iloc[i]['content']))
                parsed_tweet['sentiment_classification'] = aux_class_tweet.classification
                parsed_tweet['p_pos'] = aux_class_tweet.p_pos
                parsed_tweet['p_neg'] = aux_class_tweet.p_neg
                parsed_tweet['polarity'] = aux_class_tweet_pol_sub.polarity
                parsed_tweet['subjectivity'] = aux_class_tweet_pol_sub.subjectivity
                tweets.append(parsed_tweet)
            except KeyError as e:
                print(e)
        return tweets


def main():
    scraper = TwitterScrapeSentimentAnalysis()
    tweets = scraper.get_tweets()
    df = pd.DataFrame(tweets)
    df.to_csv('tweets_output_textBlob.csv')


if __name__ == "__main__":
    main()


Fetching tweets...
Reading file...
Reading file...


# VADER PROCESS

In [42]:
class TwitterClientSentimentAnalysisVader():

    def __init__(self):
        try:
            self.sinceDate = "2022-01-01"
            self.untilDate = "2022-03-13"
            self.hashtag = ""
            self.vaderObj = SentimentIntensityAnalyzer()
        except:
            print("Error: scraper Failed")

    def read_file(self, hashtag, since, until):
        print("Reading file...")
        os.system("snscrape --jsonl --max-results 500 --since {since} twitter-hashtag '{hashtag} until:{until}' > text-query-tweets.json".format(
            hashtag=hashtag, since=since, until=until))
        print("Reading file...")
        tweets = pd.read_json("text-query-tweets.json", lines=True)
        return tweets

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())

    def get_sentiment_tweet(self, tweet):
        translator = Translator()
        tweet = self.clean_tweet(tweet)
        tweet_en = translator.translate(tweet,dest="en")
        sentiment = self.vaderObj.polarity_scores(tweet_en.text)
        return sentiment

    def get_tweets(self):
        tweets = []
        print("Fetching tweets...")
        tweets_data = self.read_file(
            self.hashtag, self.sinceDate, self.untilDate)

        for i in range(len(tweets_data)):
            try:
                parsed_tweet = {}
                parsed_tweet['date'] = tweets_data.iloc[i]['date']
                parsed_tweet['conversationId'] = int(tweets_data.iloc[i]['conversationId'])
                parsed_tweet['tweet'] = tweets_data.iloc[i]['content']
                aux_vader_sent = self.get_sentiment_tweet(
                    str(tweets_data.iloc[i]['content']))
                parsed_tweet['compound'] = aux_vader_sent['compound']
                if aux_vader_sent['compound'] >= 0.05 :
                    parsed_tweet['classification'] = "Positive"
                elif aux_vader_sent['compound'] <= - 0.05 :
                    parsed_tweet['classification'] = "Negative"
                else :
                    parsed_tweet['classification'] = "Neutral"
                parsed_tweet['neg'] = aux_vader_sent['neg']
                parsed_tweet['neu'] = aux_vader_sent['neu']
                parsed_tweet['pos'] = aux_vader_sent['pos']
                    
                tweets.append(parsed_tweet)
            except KeyError as e:
                print(e)
        return tweets


def main():
    scraper = TwitterClientSentimentAnalysisVader()
    tweets = scraper.get_tweets()
    df = pd.DataFrame(tweets)
    df.to_csv('tweets_sentiment_vader_output.csv')


if __name__ == "__main__":
    main()


Fetching tweets...
Reading file...
Reading file...


# NRCLEX PROCESS

In [44]:
class TwitterClientSentimentAnalysisNRCLex(object):
    
    def __init__(self):
        try:
            self.sinceDate = "2022-01-01"
            self.untilDate = "2022-03-13"
            self.hashtag = ""
        except:
            print("Error: scraper Failed")

    def read_file(self, hashtag, since, until):
        print("Reading file...")
        os.system("snscrape --jsonl --max-results 500 --since {since} twitter-hashtag '{hashtag} until:{until}' > text-query-tweets.json".format(
            hashtag=hashtag, since=since, until=until))
        print("Reading file...")
        tweets = pd.read_json("text-query-tweets.json", lines=True)
        return tweets


    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())

    def get_nrclex_tweet(self, tweet):
        translator = Translator()
        tweet = self.clean_tweet(tweet)
        tweet_en = translator.translate(tweet,dest="en")
        tweetObj = NRCLex(tweet_en.text).affect_frequencies
        return tweetObj

    def get_tweets(self):
        tweets = []
        print("Fetching tweets...")
        tweets_data = self.read_file(
            self.hashtag, self.sinceDate, self.untilDate)

        for i in range(len(tweets_data)):
            try:
                parsed_tweet = {}
                parsed_tweet['date'] = tweets_data.iloc[i]['date']
                parsed_tweet['conversationId'] = int(
                    tweets_data.iloc[i]['conversationId'])
                parsed_tweet['tweet'] = tweets_data.iloc[i]['content']
                aux_NRCLex_sent = self.get_nrclex_tweet(str(tweets_data.iloc[i]['content']))
                parsed_tweet['emotion'] = max(aux_NRCLex_sent, key=aux_NRCLex_sent.get)
                parsed_tweet['score'] = aux_NRCLex_sent[max(aux_NRCLex_sent, key=aux_NRCLex_sent.get)]
                tweets.append(parsed_tweet)
            except KeyError as e:
                print(e)
        return tweets


def main():
    scraper = TwitterClientSentimentAnalysisNRCLex()
    tweets = scraper.get_tweets()
    df = pd.DataFrame(tweets)
    df.to_csv('tweets_sentiment_NRCLex_output.csv')


if __name__ == "__main__":
    main()


Fetching tweets...
Reading file...
Reading file...


# TWITTER SENTIMENT MODULE PROCESS

In [45]:
class TwitterClientSentimentAnalysisTwitterModule(object):

    def __init__(self):
        try:
            self.sinceDate = "2022-01-01"
            self.untilDate = "2022-03-13"
            self.hashtag = ""
            self.sent = TwitterSentiment.Sentiment()
            self.sent.load_pretrained()
        except:
            print("Error: Authentication Failed")

    def read_file(self, hashtag, since, until):
        print("Reading file...")
        os.system("snscrape --jsonl --max-results 500 --since {since} twitter-hashtag '{hashtag} until:{until}' > text-query-tweets.json".format(
            hashtag=hashtag, since=since, until=until))
        print("Reading file...")
        tweets = pd.read_json("text-query-tweets.json", lines=True)
        return tweets
        
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())

    def get_twitter_tweet(self, tweet):
        translator = Translator()
        tweet = self.clean_tweet(tweet)
        tweet_en = translator.translate(tweet,dest="en")
        sentiment = self.sent.predict(tweet_en.text)
        return sentiment

    def get_tweets(self):
        tweets = []
        print("Fetching tweets...")
        tweets_data = self.read_file(
            self.hashtag, self.sinceDate, self.untilDate)
        for i in range(len(tweets_data)):
            parsed_tweet = {}
            parsed_tweet['date'] = tweets_data.iloc[i]['date']
            parsed_tweet['conversationId'] = tweets_data.iloc[i]['conversationId']
            parsed_tweet['tweet'] = tweets_data.iloc[i]['content']
            aux_twitter_sent = self.get_twitter_tweet(
                str(tweets_data.iloc[i]['content']))
            if aux_twitter_sent > 0.5 :
                parsed_tweet['classification'] = "Negative"
            elif aux_twitter_sent < 0.5 :
                parsed_tweet['classification'] = "Positive"
            else :
                parsed_tweet['classification'] = "Neutral"
            parsed_tweet['score'] = aux_twitter_sent
            tweets.append(parsed_tweet)
        return tweets


def main():
    scraper = TwitterClientSentimentAnalysisTwitterModule()
    tweets = scraper.get_tweets()
    df = pd.DataFrame(tweets)
    df.to_csv('tweets_sentiment_twitter_module_output.csv')


if __name__ == "__main__":
    main()


Downloading: "https://github.com/shahules786/Twitter-Sentiment/releases/download/v1.0/classifer_model.pt" to /home/blindedsoul/.cache/torch/checkpoints/classifer_model.pt
100%|██████████| 178M/178M [00:14<00:00, 12.7MB/s] 


Fetching tweets...
Reading file...
Reading file...


# FLAIR PROCESS

In [48]:
class TwitterClientSentimentAnalysisFlair(object):
    
    def __init__(self):
        try:
            self.sinceDate = "2022-01-01"
            self.untilDate = "2022-03-13"
            self.hashtag = ""
            self.classifier = TextClassifier.load('en-sentiment')
        except:
            print("Error: Authentication Failed")


    def read_file(self, hashtag, since, until):
        print("Reading file...")
        os.system("snscrape --jsonl --max-results 500 --since {since} twitter-hashtag '{hashtag} until:{until}' > text-query-tweets.json".format(
            hashtag=hashtag, since=since, until=until))
        print("Reading file...")
        tweets = pd.read_json("text-query-tweets.json", lines=True)
        return tweets

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())

    def get_flair_tweet(self, tweet):
        translator = Translator()
        tweet = self.clean_tweet(tweet)
        tweet_en = translator.translate(tweet,dest="en")
        sentence = Sentence(tweet_en.text)
        self.classifier.predict(sentence)
        return sentence.labels[0]

    def get_tweets(self):
        tweets = []
        print("Fetching tweets...")
        tweets_data = self.read_file(
            self.hashtag, self.sinceDate, self.untilDate)
        for i in range(len(tweets_data)):
            parsed_tweet = {}
            parsed_tweet['date'] = tweets_data.iloc[i]['date']
            parsed_tweet['conversationId'] = tweets_data.iloc[i]['conversationId']
            parsed_tweet['tweet'] = tweets_data.iloc[i]['content']
            aux_flair_sent = self.get_flair_tweet(str(tweets_data.iloc[i]['content']))
            parsed_tweet['sentiment'] = str(aux_flair_sent).split(" ")[0]
            parsed_tweet['score'] = str(aux_flair_sent).split(" ")[1][1:-1]
            tweets.append(parsed_tweet)
        return tweets


def main():
    scraper = TwitterClientSentimentAnalysisFlair()
    tweets = scraper.get_tweets()
    df = pd.DataFrame(tweets)
    df.to_csv('tweets_sentiment_flair_output.csv')


if __name__ == "__main__":
    main()


2022-03-17 18:14:00,396 loading file /home/blindedsoul/.flair/models/sentiment-en-mix-distillbert_4.pt
Fetching tweets...
Reading file...
Reading file...


# TEXT2EMOTION PROCESS

In [51]:
class TwitterClientEmotionalAnalysis(object):
    
    def __init__(self):
        try:
            self.sinceDate = "2022-01-01"
            self.untilDate = "2022-03-13"
            self.hashtag = ""
        except:
            print("Error: Authentication Failed")

    def read_file(self, hashtag, since, until):
        print("Reading file...")
        os.system("snscrape --jsonl --max-results 500 --since {since} twitter-hashtag '{hashtag} until:{until}' > text-query-tweets.json".format(
            hashtag=hashtag, since=since, until=until))
        print("Reading file...")
        tweets = pd.read_json("text-query-tweets.json", lines=True)
        return tweets

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())

    def get_emotion_tweet(self, tweet):
        translator = Translator()
        tweet = self.clean_tweet(tweet)
        tweet_en = translator.translate(tweet,dest="en")
        emotions = te.get_emotion(tweet_en.text)
        return emotions

    def get_tweets(self):
        tweets = []
        print("Fetching tweets...")
        tweets_data = self.read_file(
            self.hashtag, self.sinceDate, self.untilDate)
        for i in range(len(tweets_data)):
            try:
                parsed_tweet = {}
                parsed_tweet['date'] = tweets_data.iloc[i]['date']
                parsed_tweet['conversationId'] = tweets_data.iloc[i]['conversationId']
                parsed_tweet['tweet'] = tweets_data.iloc[i]['content']
                aux_emotions = self.get_emotion_tweet(str(tweets_data.iloc[i]['content']))
                parsed_tweet['Angry'] =  aux_emotions['Angry']
                parsed_tweet['Fear'] =  aux_emotions['Fear']
                parsed_tweet['Happy'] =  aux_emotions['Happy']
                parsed_tweet['Sad'] =  aux_emotions['Sad']
                parsed_tweet['Surprise'] =  aux_emotions['Surprise']
                tweets.append(parsed_tweet)
            except KeyError  as e:
                print(e)
        return tweets


def main():
    scraper = TwitterClientEmotionalAnalysis()
    tweets = scraper.get_tweets()
    df = pd.DataFrame(tweets)
    df.to_csv('tweets_emotions_text2emotions_output.csv')


if __name__ == "__main__":
    main()


Fetching tweets...
Reading file...
Reading file...
